In [1]:
import pandas as pd
import spacy
import numpy as np

nlp = spacy.load("fr_core_news_sm")

# Objectif : trouver la question du thème la plus similaire

In [256]:
theme_quest_user = "Préparer mon séjour" # on fixe un theme au pif

In [257]:
#quest_user = "Puis-je évaluer le confort de mon domaine ?"
#quest_user = "Les annimaux domestiques sont-ils acceptés ?"
quest_user = "ai-je besoin d'un visa ou d'un passeport ?"

In [258]:
faq = pd.read_pickle('faq_centerPark.pkl')

In [260]:
faq_theme

,question,reponse
0,Quels sont les jours d’arrivée ?,Il est possible d’arriver n’importe quel jour ...
1,Comment évaluer le confort de mon domaine et d...,Le classement par « birdies » évalue l’offre C...
2,Quels sont les services et activités compris d...,"En réservant votre hébergement, vous bénéficie..."
3,Comment réserver mes activités ?,"Lors de la réservation de votre hébergement, v..."
4,Où trouver le plan du domaine ?,"Sur la page d'accueil de notre site, cliquez s..."
5,Puis-je venir avec mon animal domestique ?,La présence de votre animal domestique est aut...
6,Peut-on accéder au domaine à la journée (sans ...,Il est possible de passer la journée à Center ...
7,Comment réserver un logement adapté aux person...,"Tous nos domaines français, dont Villages Nat..."
8,Comment recevoir la brochure ?,Cliquez ici pour remplir une demande d'envoi d...
9,Est-il possible d'acheter des billets pour les...,Oui. Achetez directement vos billets lors de v...


### Nettoyage du corpus

In [261]:
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer('french')

sw = nltk.corpus.stopwords.words('french')
sw += ['être', 'avoir']
sw.sort()

def lemmatise_text(text):
    lst_lematised = [token.lemma_ for token in nlp(text)] 
    return ' '.join(lst_lematised).lower()


def stem_text(text):
    lst_stemmerised = [stemmer.stem(token) for token in word_tokenize(text)]    
    return ' '.join(lst_stemmerised)


def substitute_punctuation(text):
    return ' '.join(text.replace("'", ' ').translate(str.maketrans('', '', string.punctuation)).split())


def supp(text):
    return text.replace("«", "").replace("’", "").replace("•", "").replace("®", "")


def supp_sw(text):
    return ' '.join([token.text for token in nlp(text) if not token.text in sw])

In [262]:
faq_theme['question_clean'] = (faq_theme.question.apply(lemmatise_text)
                                                 .apply(stem_text)
                                                 .apply(substitute_punctuation)
                                                 .apply(supp)
                                                 .apply(supp_sw)
                              )
faq_theme['reponse_clean'] = (faq_theme.reponse.apply(lemmatise_text)
                                               .apply(stem_text)
                                               .apply(substitute_punctuation)
                                               .apply(supp)
                                               .apply(supp_sw)
                              )
faq_theme['tokens'] = faq_theme['question_clean'].apply(nlp)

In [263]:
faq_theme.to_pickle('df_classif_similarity.pkl')

### sélection des questions du thème

In [259]:
faq_theme = faq[faq.theme == theme_quest_user][["question", 'reponse']]

### Nettoyage de la question

In [249]:
quest_user_clean = supp_sw(supp(substitute_punctuation(stem_text(lemmatise_text(quest_user)))))

### Similarité

In [250]:
quest_user_clean_tokens = nlp(quest_user_clean)

In [251]:
lst_similarity = [quest_user_clean_tokens.similarity(token) for token in faq_theme.tokens]

In [252]:
rep_quest_user = faq_theme.reponse[np.asarray(lst_similarity).argmax()]

In [253]:
rep_quest_user

'Il est possible d’arriver n’importe quel jour de la semaine, à l’exception du dimanche, et pour une durée de 2 à 7 nuits. Les possibilités d’arrivées pendant les périodes de vacances scolaires sont plus limitées. Vous pouvez vous référer aux possibilités d’arrivées et de séjour dans votre réservation.'